<a href="https://colab.research.google.com/github/gionata/optimization-notebooks/blob/main/shortestPath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q amplpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 39.0 MB/s eta 0:00:00


In [2]:
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["snopt"],
    license_uuid="default"
)

In [3]:
%%writefile shortestPath.mod
set INTERSECTIONS;
param source symbolic in INTERSECTIONS;
param destination symbolic in INTERSECTIONS;

set ROADS within
    (INTERSECTIONS diff { destination })
      cross
    (INTERSECTIONS diff { source });

param cost { ROADS } >= 0;
var Use { ROADS} >= 0, <= 1; # binary: {0, 1}

minimize Total_Cost:
  sum {(i, j) in ROADS} cost[i, j] * Use[i, j];

subject to Start:
  sum {(source, j) in ROADS} Use[source, j] = 1;

subject to Balance {k in INTERSECTIONS diff { source, destination }}:
  sum {(i, k) in ROADS} Use[i, k] =
    sum {(k, j) in ROADS} Use[k, j];

Writing shortestPath.mod


In [4]:
%%writefile shortestPath.dat
set INTERSECTIONS := a b c d e f g ;
param source := a ;
param destination := g ;
param: ROADS: cost :=
    a b 50, a c 100
    b d 40, b e 20
    c d 60, c f 20
    d e 50, d f 60
    e g 70, f g 70 ;

Writing shortestPath.dat


In [11]:
ampl.reset()
ampl.read('shortestPath.mod')
ampl.readData('shortestPath.dat')
ampl.eval('show;')
ampl.eval('expand;')
ampl.solve(solver="snopt")
ampl.eval('display {(i, j) in ROADS : Use[i, j] >= 0.9} Use[i,j];')


parameters:   cost   destination   source

sets:   INTERSECTIONS   ROADS

variable:   Use

constraints:   Balance   Start

objective:   Total_Cost
minimize Total_Cost:
	50*Use['a','b'] + 100*Use['a','c'] + 40*Use['b','d'] + 20*Use['b','e']
	 + 60*Use['c','d'] + 20*Use['c','f'] + 50*Use['d','e'] + 
	60*Use['d','f'] + 70*Use['e','g'] + 70*Use['f','g'];

subject to Start:
	Use['a','b'] + Use['a','c'] = 1;

subject to Balance['b']:
	Use['a','b'] - Use['b','d'] - Use['b','e'] = 0;

subject to Balance['c']:
	Use['a','c'] - Use['c','d'] - Use['c','f'] = 0;

subject to Balance['d']:
	Use['b','d'] + Use['c','d'] - Use['d','e'] - Use['d','f'] = 0;

subject to Balance['e']:
	Use['b','e'] + Use['d','e'] - Use['e','g'] = 0;

subject to Balance['f']:
	Use['c','f'] + Use['d','f'] - Use['f','g'] = 0;

SNOPT 7.5-1.2 : Optimal solution found.
1 iterations, objective 140
Use[i,j] :=
a b   1
b e   1
e g   1
;

